In [4]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/20201.ai4eng/master/init.py
import init; init.init(force_download=False); init.get_weblink()


enabling encryption...
encryption enabled
replicating local resources
endpoint https://m5knaekxo6.execute-api.us-west-2.amazonaws.com/dev-v0001/rlxmooc


In [5]:
from local.lib.rlxmoocapi import submit, session
student = session.Session(init.endpoint).login( course_id=init.course_id, 
                                                session_id="UDEA", 
                                                lab_id="L02.03" )
init.get_weblink()

username: davida.torres@udea.edu.co
password: ··········
endpoint https://m5knaekxo6.execute-api.us-west-2.amazonaws.com/dev-v0001/rlxmooc


In [6]:
import numpy as np
import pandas as pd

# Task 1: Extract data

we have a dataframe of items with a price and, **sometimes**, with an extra column (`margin`).

You will have to complete a function that will **filter** the dataframe selecting the rows:

- whose price is > 100
- **OR** whose margin is >10, if the column `margin` is present

your function must return **A LIST** with the item ids of the selected rows

For instance, with the following dataframe

             price  category
    itemid                  
    56556   108.15       2.0
    73065    83.54       2.0
    36619   114.42       0.0
    73414    82.94       0.0
    13410   115.13       2.0
    66153    91.50       1.0
    77380    85.82       0.0
    73249    95.95       0.0
    11654   100.22       2.0
    11972    77.16       1.0

your must return the following list:

     [56556, 36619, 13410, 11654]
     
But if you get the following dataframe

             price  category  margin
    itemid                          
    39059    98.11       0.0   11.04
    19526    98.11       1.0   11.25
    78176    94.34       1.0   10.51
    50948   102.37       1.0   10.77
    12111    98.07       1.0    8.50
    56191    98.53       1.0   11.65
    38887    91.49       2.0   11.24
    77915   117.30       0.0    8.64
    55010    96.13       0.0    8.95
    45925    98.59       1.0   10.45
    
you must return the following list

    [39059, 19526, 78176, 50948, 56191, 38887, 77915, 45925]

In [7]:
def create_df(missing=False, n=10):
    itemid   = np.random.randint(100000, size=n)+1000
    category = np.random.randint(3, size=n)
    price    = np.round(np.random.normal(loc=100, scale=10, size=n),2)
    margin   = np.round(np.random.normal(loc=10, scale=1, size=n),2)
    
    if missing:
        nmissing = np.random.randint(len(price)//2)+2                                     
        price[np.random.permutation(len(price))[:nmissing]] = np.nan
    
    d = pd.DataFrame(np.r_[[price, category, margin]].T, index=itemid, columns=["price", "category", "margin"])
    d.index.name="itemid"
    if np.random.random()>.5:
        d = d[d.columns[:2]]
        
    return d

In [8]:
d = create_df()
d

,price,category
itemid,,
43953,99.32,1.0
26618,99.34,1.0
94469,97.81,1.0
42524,104.71,0.0
90802,80.83,2.0
43431,109.60,1.0
31181,92.60,1.0
5914,97.49,1.0
55371,100.99,1.0


In [9]:
def select_items(df):
  if len(df.columns)<3:
    return  df[df.price>100].index.tolist()
  return df[(df.price>100) | (df.margin>10)].index.tolist()

**manually check your answer**

In [10]:
d = create_df()
d

,price,category
itemid,,
26649,100.67,0.0
22890,106.25,0.0
100248,97.85,2.0
46583,124.34,1.0
83238,102.97,1.0
52085,99.71,2.0
81302,112.13,0.0
7873,97.05,1.0
66638,99.07,1.0


In [11]:
select_items(d)

[26649, 22890, 46583, 83238, 81302]

**submit your code**

In [12]:
student.submit_task(globals(), task_id="task_01");

# Task 2: Group statistics

Complete the following function so that it returns a dataframe with the average, max and min **prices** per category. 

For instance, for the following dataframe

             price  category  margin
    itemid                          
    17946    93.85       1.0   10.64
    61190    91.72       1.0    9.76
    39639   100.16       1.0   10.67
    17791   110.44       2.0    9.65
    7333    101.05       1.0    9.69
    77362   122.33       0.0   11.14
    92646   108.13       2.0   10.58
    27797    85.52       2.0   10.88
    31746    97.56       0.0    9.75
    12355   101.04       2.0    9.51
    
you should return the following dataframe

                 media  maximo  minimo
    categoria                         
    0         109.9450  122.33  97.56
    1          96.6950  101.05  91.72
    2         101.2825  110.44  85.52
    
observe that your result
- must not be a **multilevel** columnset.
- the column names and the index name must be **exactly** as in the example.
- the **index** must be of type **int**.

In [ ]:
def get_stats(df):

  #Group by category
  group=df.groupby("category").price
  joingroup=[group.mean(),group.max(),group.min()]
  result=pd.concat(joingroup,axis=1)

  #Change colums names and index name 
  result.columns=["media","maximo","minimo"]
  result.index=result.index.astype(int) # Change Index typeof
  result.index.names=['categoria']
  return result


**manually check your answer**

In [ ]:
d = create_df()



In [ ]:
get_stats(d)

,media,maximo,minimo
categoria,,,
0,105.385000,106.37,104.40
1,96.793333,118.97,82.44
2,100.170000,112.58,89.48


**submit your code**

In [ ]:
student.submit_task(globals(), task_id="task_02");

# Task 3: Fill in missing data

Fill in the missing data in the **price** column with the following procedure:

- compute the mean and std of the avaialable prices
- sample from a normal distribution with the computed mean and std (see [`np.random.normal`])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
- substitute the missing values with the samples

For instance, for this input dataframe:

             price  category  margin
    itemid                          
    18922      NaN       1.0   10.32
    69500   121.25       1.0   10.22
    76442    90.25       1.0   12.60
    33863   106.51       0.0   10.26
    15904    95.87       1.0   11.51
    41946   103.47       2.0    9.85
    85451    93.08       2.0    9.56
    70028   116.68       1.0    9.11
    26860      NaN       2.0    9.71
    12807    91.48       0.0    9.77
    
your solution might be similar to this (not exactly the same as you will be sampling data):


                 price  category  margin
    itemid                              
    18922    97.441188       1.0   10.32
    69500   121.250000       1.0   10.22
    76442    90.250000       1.0   12.60
    33863   106.510000       0.0   10.26
    15904    95.870000       1.0   11.51
    41946   103.470000       2.0    9.85
    85451    93.080000       2.0    9.56
    70028   116.680000       1.0    9.11
    26860   103.294843       2.0    9.71
    12807    91.480000       0.0    9.77
    
    
**WARN**: your function must not modify the original dataframe, make a copy of the input dataframe, fill the values in the copy and return it.

**HINT**: use the [`isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) method of a dataframe or a series.

In [13]:
d = create_df(missing=True)
d

,price,category
itemid,,
94933,99.81,2.0
38468,NaN,2.0
59332,97.83,0.0
2006,NaN,0.0
70001,113.70,2.0
15775,119.33,0.0
12161,103.42,2.0
78353,103.19,2.0
81024,96.72,2.0


In [36]:
'''
Fill in the missing data in the price column with the following procedure:

compute the mean and std of the avaialable prices
sample from a normal distribution with the computed mean and std (see [np.random.normal])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
substitute the missing values with the samples
'''
def fillna(df):
  result = df.copy()
  label="price"
  missingCondition= result["price"].isna() == True
  _mean= df["price"].mean()
  _std = df["price"].std()
  _size=len(result["price"].isna() ==True)
  result[ label][ missingCondition ] = np.random.normal(_mean,_std,_size)
  return result

**check manually your code**

In [37]:
d = create_df(missing=True)
d

,price,category,margin
itemid,,,
69345,100.29,2.0,10.14
59125,100.37,2.0,11.39
15938,NaN,1.0,11.25
75061,114.73,0.0,9.44
92147,NaN,0.0,8.84
83466,NaN,0.0,9.58
18047,NaN,2.0,10.06
69095,NaN,1.0,10.08
1595,105.28,1.0,10.18


In [38]:
fillna(d)



,price,category,margin
itemid,,,
69345,100.290000,2.0,10.14
59125,100.370000,2.0,11.39
15938,110.464729,1.0,11.25
75061,114.730000,0.0,9.44
92147,102.851302,0.0,8.84
83466,97.003249,0.0,9.58
18047,110.016192,2.0,10.06
69095,93.624513,1.0,10.08
1595,105.280000,1.0,10.18


price       92.822528
category     0.400000
margin       9.844000
dtype: float64

**submit your code**

In [39]:
student.submit_task(globals(), task_id="task_03");